In [ ]:
import asyncio
import websockets
import json
import requests
import os
import dotenv
from dotenv import load_dotenv

load_dotenv()

def send_post_request(url, text_data):
    data = {"text": text_data}
    response = requests.post(url, json=data)
    return response

async def websocket_listen(uri, msg):
    async with websockets.connect(uri) as websocket:
        await websocket.send(json.dumps(msg))
        async for message in websocket:
            if "AudioSummaryLink" in message:
                data = json.loads(message)
                print(f"AudioSummaryLink:\n{data['AudioSummaryLink']}")
                print()
                print(f"SummaryText:\n{data['SummaryText']}") 
                await websocket.close()
                break

def read_text_data(filename="sample_text_1000_tokens.txt"):
    with open(filename, 'r') as file:
        text = file.read()
    return text


In [ ]:
web_socket_uri = os.getenv('WEBSOCKET_URI')
url = os.getenv('API_URL')
msg_for_websocket = {"Action": "register", "executionArn": "test-execution"}
text = read_text_data()

In [ ]:
response = send_post_request(url, text)
msg = {"Action": "register", 
       "executionArn": f"{response.json().get('executionArn')}"}
await websocket_listen(web_socket_uri, msg)